# Repertoire

In [1]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab :(")

Mounted at /gdrive


In [2]:
REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

In [ ]:
import pickle
with open(REP_INTERMED + 'dict_words.pkl', 'rb') as f:
        dict_words = pickle.load(f) 

In [ ]:
dict_words

# MODULES

In [ ]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [ ]:
%%capture
! pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from functools import partial

# FUNCTIONS

In [ ]:
%run "00_functions.ipynb"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Inputs

In [ ]:
baseline = pd.read_pickle(REP_INTERMED + 'baseline.sav').drop([	'released_date'], axis=1) 
baseline['text_2'] = baseline['text_CLEAN'].str.title()

In [ ]:
df = pd.read_pickle(REP_INTERMED + 'base_cdandlp.sav')
df['text']  = df['artiste']  + ' ' +  df['titre']

# Dict of words

In [ ]:
baseline['text_2'] = baseline['text_CLEAN'].str.title()
# stop_words = ['and', 'et']
# dict_words = {}


for id_row,text in enumerate(baseline['text_2']):
    words = text.split()
    words = [ w for w in words if w not in stop_words ]
    words = set(words)
    for word in words:
        if word not in dict_words:
            dict_words[word] = [ id_row ]
        else:
            liste_temp = dict_words[word]
            liste_temp.append(id_row)
            dict_words[word] = liste_temp
  
 with open(REP_INTERMED + 'dict_words.pkl', 'wb') as f:
        pickle.dump(dict_words, f) 

with open(REP_INTERMED + 'dict_words.pkl', 'rb') as f:
        dict_words = pickle.load(f) 

# Modeling

## Based on knn from scratch

In [ ]:
start_time = datetime.now()
row = df.loc[0, 'text']
# retrieve the list of indices by word
indices_list = [dict_words.get(key) for key in row.split()]
while None in indices_list:
    indices_list.remove(None)
# convert the list of lists to flat list
indices_list_flat = list(set([item for sublist in indices_list for item in sublist]))
# match indexes with baseline indexes
df_baseline = baseline.iloc[indices_list_flat,].reset_index(drop=True)
answer = knn_words(df_baseline, df.iloc[[0,]])
#pd.concat(answer)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:

answer = knn_words(df_baseline, df.iloc[[0,]])
#pd.concat(answer)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

<function __main__.knn_words>

In [ ]:
start_time = datetime.now()
row = df.loc[2, 'text']
# retrieve the list of indices by word
indices_list = [dict_words.get(key) for key in row.split()]
while None in indices_list:
    indices_list.remove(None)
# convert the list of lists to flat list
indices_list_flat = list(set([item for sublist in indices_list for item in sublist]))
# match indexes with baseline indexes
df_baseline = baseline.iloc[indices_list_flat,].reset_index(drop=True)
df_sample = df.iloc[[2,]].reset_index(drop='True')
answer = knn_words(df_baseline, df_sample)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
pd.concat(answer)


Duration: 0:00:10.635166


,id_query,master_id,artist,title,similarity,rank,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial
0,117183052,117183052,Patrick Moraz,Future Memories : Patrick Moraz Live On Tv,-1.0,0,1,1,1,1,1,1
1,117183052,"18058,0",Patrick Moraz,Future Memories,88.396,1,0.675325,0.901961,0.509804,1.0,0.509804,1.0
2,117183052,18025,Patrick Moraz,Patrick Moraz,84.407222,2,0.692308,0.868845,0.529412,1.0,0.352941,0.74
3,117183052,18044,Patrick Moraz,Future Memories Ii,82.686887,3,0.725,0.913725,0.568627,1.0,0.529412,0.93
4,117183052,0,Patrick Moraz,"Patrick Moraz 1,2",82.591801,4,0.707317,0.852546,0.54717,0.935484,0.352941,0.71
5,117183052,18025,Patrick Moraz,Patrick Moraz Iii,80.075722,5,0.707317,0.864529,0.576923,0.967742,0.352941,0.71
6,117183052,600889,Patrick Moraz,Future Memories I & Ii,76.897243,6,0.731707,0.911195,0.576923,0.967742,0.529412,0.87
7,117183052,18058,Patrick Moraz,Future Memories Live On Tv (Keyboards' Metamor...,76.338102,7,0.678899,0.912086,0.786885,0.941176,0.5,0.73
8,117183052,758817,Patrick Moraz,Resonance,73.854895,8,0.514286,0.845614,0.346154,0.947368,0.27451,0.79
9,117183052,0,Patrick Moraz,Karu,73.098799,9,0.463768,0.848366,0.352941,1.0,0.294118,0.83


## Based on text distances

In [ ]:
start_time = datetime.now()
row = df.loc[0, 'text']
# retrieve the list of indices by word
indices_list = [dict_words.get(key) for key in row.split()]
indices_list.remove(None)
# convert the list of lists to flat list
indices_list_flat = list(set([item for sublist in indices_list for item in sublist]))
# match indexes with baseline indexes
df_baseline = baseline.iloc[indices_list_flat,].reset_index(drop=True)
df_baseline['text_query'] = make_text_prep_func(row)
df_baseline['index'] = df_baseline.index
# KNN based on text distances
answer2 = knn_words_distance(0, df_baseline, df.iloc[[0,]], 20)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
answer2

Duration: 0:02:51.078747


,id_query,master_id,artist,title,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial,rank
1,113543790,237773,Francis Cabrel,Les Chemins De Traverse,0.909091,0.966667,0.833333,1.0,0.833333,1.0,1.0
2,113543790,110748,Francis Cabrel,Je L'Aime A Mourir / Les Chemins De Traverse,0.725275,0.84613,0.685185,0.880952,0.122449,0.68,2.0
3,113543790,74735,Francis Cabrel,L'Encre De Tes Yeux,0.702703,0.877966,0.644444,0.90625,0.404762,0.72,3.0
4,113543790,0,Francis Cabrel,La Chica Que Me Acompaña,0.7,0.878519,0.632653,0.815789,0.404762,0.68,4.0
5,113543790,0,Francis Cabrel,Le Chêne Liège,0.695652,0.896233,0.568182,0.925926,0.380952,0.85,5.0
6,113543790,0,Francis Cabrel,3.Cd (Les Murs De Poussière-Les Chemins De Tra...,0.692308,0.829469,0.6,0.928571,0.177419,0.62,6.0
7,113543790,0,Francis Cabrel,Les Gens Absents,0.685714,0.894139,0.590909,0.928571,0.428571,0.79,7.0
8,113543790,1378147,Francis Cabrel,Le Monde Est Sourd,0.684932,0.878264,0.622222,0.903226,0.404762,0.74,8.0
9,113543790,886082,Francis Cabrel,Qu'Est-Ce Que T'En Dis ?,0.666667,0.85093,0.591837,0.805556,0.404762,0.69,9.0
10,113543790,308107,Francis Cabrel,Le Premier Disque,0.666667,0.874066,0.565217,0.866667,0.428571,0.73,9.0


## Based on sklearn knn

In [ ]:
start_time = datetime.now()
k=20
row = df_sample.loc[37, 'text']
df_sample=df_sample.iloc[[37,]]
# retrieve the list of indices by word
indices_list = [dict_words.get(key) for key in row.split()]
indices_list.remove(None)
# convert the list of lists to flat list
indices_list_flat = list(set([item for sublist in indices_list for item in sublist]))
# match indexes with baseline indexes
df_baseline = baseline.iloc[indices_list_flat,].reset_index(drop=True)
if len(df_baseline) < k :
  k =len(df_baseline)
# train
vectorizer = TfidfVectorizer(analyzer = 'char', ngram_range=(2,2) , use_idf = False)
X_train = vectorizer.fit_transform(df_baseline['text_CLEAN'].str.replace(' ', ''))
# KNN with sklearn api
nbrs = NearestNeighbors(n_neighbors=k, metric="cosine",algorithm='brute').fit(X_train)
result = nbrs.kneighbors(text_to_vect(df_sample.loc[37,'text'], vectorizer))
# customize output
distances, indices = result
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
customize_output_blocking(37,results= result, df_baseline=df_baseline.copy(), sample=df_sample)

Duration: 0:00:00.017366


,id_query,master_id,artist,title,distance,rank,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial
0,111766837,111766837,Atoll,Musiciens-Magiciens,-1.0,0,1,1,1,1,1,1
1,111766837,0,Various,Iles Et Atolls Du Pacifique,0.649562,1,0.416667,0.582085,0.411765,0.636364,0.038462,0.47
2,111766837,0,Coup D'Marron,Atoll À Terre,0.765854,2,0.291667,0.47972,0.333333,0.545455,0.115385,0.32
3,111766837,0,Various,Melodies Des Atolls,0.783705,3,0.318182,0.558081,0.419355,0.590909,0.045455,0.32
4,111766837,0,François Saint-Alme Et Atoll (2),Pays Bon Dieu,0.797223,4,0.354839,0.575713,0.409091,0.818182,0.075,0.41
5,111766837,0,Atoll,Le Secret Du Mage / Je Fais Un Rêve,0.797223,4,0.474576,0.636568,0.372093,0.727273,0.108108,0.55
6,111766837,0,Maurice Bitter,Nouvelle Calédonie Et Sa Couronne - Ouvea Atol...,0.822295,6,0.315789,0.569024,0.357143,0.909091,0.037037,0.41


# Level 1 running

In [ ]:
col_cdandlp = ['pays_clean', 'text', 'n_ref', 'artiste', 'titre']
with open(REP_INTERMED + 'dict_country_cdandlp.pkl', 'rb') as f:
  dict_country_cdandlp = pickle.load(f)  

df_sample= pd.DataFrame(dict_country_cdandlp['France'], columns=col_cdandlp)

In [ ]:
level1_knn_words(37, df_sample, baseline, dict_words)

37


,id_query,master_id,artist,title,similarity,rank,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial
0,111766837,111766837,Atoll,Musiciens-Magiciens,-1.0,0,1,1,1,1,1,1
1,111766837,0,Various,Iles Et Atolls Du Pacifique,35.043832,1,0.416667,0.582085,0.411765,0.636364,0.038462,0.47
2,111766837,0,Coup D'Marron,Atoll À Terre,23.414645,2,0.291667,0.47972,0.333333,0.545455,0.115385,0.32
3,111766837,0,Various,Melodies Des Atolls,21.629523,3,0.318182,0.558081,0.419355,0.590909,0.045455,0.32
4,111766837,0,François Saint-Alme Et Atoll (2),Pays Bon Dieu,20.277678,4,0.354839,0.575713,0.409091,0.818182,0.075,0.41
5,111766837,0,Atoll,Le Secret Du Mage / Je Fais Un Rêve,20.277678,4,0.474576,0.636568,0.372093,0.727273,0.108108,0.55
6,111766837,0,Maurice Bitter,Nouvelle Calédonie Et Sa Couronne - Ouvea Atol...,17.770466,6,0.315789,0.569024,0.357143,0.909091,0.037037,0.41


In [ ]:
result = [ level1_knn_words(indice, df_sample, baseline, dict_words)  for indice in range(len(df_sample)) ]

In [ ]:
df = df_sample
baseline
index =37
row = df.loc[index, 'text']
  # retrieve the list of indices by word
indices_list = [dict_words.get(key) for key in row.split()]
distances = ['levenshtein', 'jaro_winkler', 'jaccard', 'overlap', 'hamming', 'fuzzy_partial']
while None in indices_list:
      indices_list.remove(None)
# convert the list of lists to flat list
indices_list_flat = list(set([item for sublist in indices_list for item in sublist]))
df_baseline = baseline.iloc[indices_list_flat,].reset_index(drop=True)
df_sample = df.iloc[[index,]].reset_index(drop='True')

In [ ]:
def top_k(x, k):
        ind = np.argpartition(x, -1 * k)[-1 * k:]                  
        return ind[np.argsort(x[ind])]                    

topsim = np.apply_along_axis(lambda x: top_k(x, nb_top), 0, mat_sim)
topsim2 = pd.DataFrame( topsim, columns=['index'])

l_sim = []
for i in list(topsim2['index']):
    l_sim.append( float(mat_sim[i])*100.0 )
topsim2['similarity'] = l_sim

In [ ]:
nb_top

20

In [ ]:
 
   #recupération des infos d'origine
df_nearest = pd.merge(baseline, topsim2, on = 'index', how='inner')

df_nearest["rank"] =  df_nearest['similarity'].rank(method ='min',ascending = False).astype(int)

df_nearest['id_query'] =sample.loc[indice,'n_ref']

df_nearest['text_query'] = make_text_prep_func(sample.loc[indice,'text'])

distances = ['levenshtein', 'jaro_winkler', 'jaccard', 'overlap', 'hamming', 'fuzzy_partial']

col_list_values = df_nearest.apply(lambda p:compound_similarity(p, "text_CLEAN", "text_query"), axis=1)
    
df_nearest.drop(['index','text_query', 'text_CLEAN'], axis=1, inplace=True)

df_nearest = pd.concat([
                            df_nearest[['id_query', 'master_id', 'artist', 'title', 'similarity', 'rank']], 
                            pd.DataFrame(np.column_stack(col_list_values).T, columns=distances )], 
                          axis= 1
                          )

df_nearest.sort_values(by='similarity', ascending=False, inplace=True)
new_row = pd.DataFrame(np.array([[
                                    sample.loc[indice,  'n_ref'],
                                    sample.loc[indice,  'n_ref'], 
                                    sample.loc[indice, 'artiste'],
                                    sample.loc[indice, 'titre'],
                                    float(-1),
                                    0,
                                    1,
                                    1,
                                    1,
                                    1,
                                    1,
                                    1,
                                  ]]
                                  ), 
                        columns= ['id_query', 'master_id', 'artist', 'title', 'similarity', 'rank'] + distances
                        )

df_nearest = new_row.append(df_nearest, ignore_index=True)

# Fin

In [ ]:
"""# Inputs"""
baseline = pd.read_pickle(REP_INTERMED + 'baseline.sav').drop([	'released_date'], axis=1)

with open(REP_INTERMED + 'dict_words.pkl', 'rb') as f:
  dict_words = pickle.load(f)

with open(REP_INTERMED + 'dict_country_cdandlp.pkl', 'rb') as f:
  		dict_country_cdandlp = pickle.load(f)
    
dict_country_cdandlp_france = dict_country_cdandlp['France']


In [ ]:
def knn(start , end, baseline = baseline, dict_words=dict_words, dict_country_cdandlp =dict_country_cdandlp_france ) :

  df_sample= pd.DataFrame(dict_country_cdandlp_france[start: end], columns=['pays_clean', 'text', 'n_ref', 'artiste', 'titre'])

  result = [ level1_knn_words(indice, df_sample, baseline, dict_words)  for indice in range(len(df_sample)) ]
  
  with open(REP_INTERMED + str(end) + '_France_words_knn.pkl', 'wb') as  f1:
    pickle.dump(result, f1) 

In [ ]:
taille = len(dict_country_cdandlp_france)
pas = int(taille/15)
list_tmp = [ indices for indices in range(0,  taille, pas) ]

In [ ]:
 # start
start_time = datetime.now()

[ knn(start, end) for start , end in zip(list_tmp,list_tmp[1:])]

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
  with open(REP_INTERMED + str(1) + '_France_words_knn.pkl', 'rb') as  f1:
    tmp = pickle.load(f1) 

In [ ]:
pd.concat(tmp)

,id_query,master_id,artist,title,similarity,rank,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial
0,117183052,117183052,Patrick Moraz,Future Memories : Patrick Moraz Live On Tv,-1.0,0,1,1,1,1,1,1
1,117183052,"18058,0",Patrick Moraz,Future Memories,88.396,1,0.675325,0.901961,0.509804,1.0,0.509804,1.0
2,117183052,18025,Patrick Moraz,Patrick Moraz,84.407222,2,0.692308,0.868845,0.529412,1.0,0.352941,0.74
3,117183052,18044,Patrick Moraz,Future Memories Ii,82.686887,3,0.725,0.913725,0.568627,1.0,0.529412,0.93
4,117183052,0,Patrick Moraz,"Patrick Moraz 1,2",82.591801,4,0.707317,0.852546,0.54717,0.935484,0.352941,0.71
5,117183052,18025,Patrick Moraz,Patrick Moraz Iii,80.075722,5,0.707317,0.864529,0.576923,0.967742,0.352941,0.71
6,117183052,600889,Patrick Moraz,Future Memories I & Ii,76.897243,6,0.731707,0.911195,0.576923,0.967742,0.529412,0.87
7,117183052,18058,Patrick Moraz,Future Memories Live On Tv (Keyboards' Metamor...,76.338102,7,0.678899,0.912086,0.786885,0.941176,0.5,0.73
8,117183052,758817,Patrick Moraz,Resonance,73.854895,8,0.514286,0.845614,0.346154,0.947368,0.27451,0.79
9,117183052,0,Patrick Moraz,Karu,73.098799,9,0.463768,0.848366,0.352941,1.0,0.294118,0.83
